<a href="https://colab.research.google.com/github/dmuman/ML-labs/blob/main/lab3/lab3_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [252]:
import numpy as np
import pandas as pd
# import

# 1

In [253]:
url = 'https://raw.githubusercontent.com/OlhaOsypova/Machine-Learning-and-Data-Analytics/refs/heads/main/Topic_4_regression/Task/clients_LTV.csv'

df = pd.read_csv(url)

df.head()

,Unnamed: 0,Customer_ID,Age,Location,Income_Level,Total_Transactions,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Total_Spent,...,Last_Transaction_Days_Ago,Loyalty_Points_Earned,Referral_Count,Cashback_Received,App_Usage_Frequency,Preferred_Payment_Method,Support_Tickets_Raised,Issue_Resolution_Time,Customer_Satisfaction_Score,LTV
0,0,cust_0000,54,Urban,Low,192,16736.38402319893$,60216.83450988285$,6525.814860765247$,3.213386e+06,...,209,2114.0,25,2224.012140,Monthly,Debit Card,3,61.568590,1,3.279546e+05
1,1,cust_0001,67,Suburban,High,979,14536.734682848608$,48350.10027249861$,2186.742245256916$,1.423146e+07,...,240,2960.0,20,4026.823518,Monthly,UPI,17,60.392889,8,1.437053e+06
2,2,cust_0002,44,Urban,High,329,7061.372800216672$,32521.157187328816$,2743.4068078554915$,NaN,...,21,3170.0,0,1441.011395,NaN,Debit Card,11,45.305579,4,2.419387e+05
3,3,cust_0003,30,Rural,High,71,16426.876453277957$,17827.896720459485$,4360.78499399125$,NaN,...,285,NaN,35,4365.855580,NaN,Wallet Balance,6,22.030191,1,1.284599e+05
4,4,cust_0004,58,Urban,Middle,878,10800.09265965521$,17497.634533762135$,4532.872520080136$,9.482481e+06,...,329,1992.0,18,4161.523827,Daily,UPI,18,20.634723,5,9.569514e+05


In [254]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   7000 non-null   int64  
 1   Customer_ID                  7000 non-null   object 
 2   Age                          7000 non-null   int64  
 3   Location                     6650 non-null   object 
 4   Income_Level                 6510 non-null   object 
 5   Total_Transactions           7000 non-null   int64  
 6   Avg_Transaction_Value        7000 non-null   object 
 7   Max_Transaction_Value        7000 non-null   object 
 8   Min_Transaction_Value        7000 non-null   object 
 9   Total_Spent                  5950 non-null   float64
 10  Active_Days                  7000 non-null   int64  
 11  Last_Transaction_Days_Ago    7000 non-null   int64  
 12  Loyalty_Points_Earned        6370 non-null   float64
 13  Referral_Count    

# 2

In [255]:
df.drop(['Unnamed: 0', 'Customer_ID', 'Location'], axis=1, inplace=True)

In [256]:
regex = r'[^\d\.]'

columns_to_change = ['Avg_Transaction_Value', 'Max_Transaction_Value', 'Min_Transaction_Value']

for column in columns_to_change:
  print(df[column][df[column].str.contains(regex, na=False)])

0        16736.38402319893$
1       14536.734682848608$
2        7061.372800216672$
3       16426.876453277957$
4        10800.09265965521$
               ...         
6995    162.70350256920446$
6996    1575.7078765992385$
6997     14429.35517756313$
6998    8005.0271841612175$
6999       15697.668867092$
Name: Avg_Transaction_Value, Length: 7000, dtype: object
0        60216.83450988285$
1        48350.10027249861$
2       32521.157187328816$
3       17827.896720459485$
4       17497.634533762135$
               ...         
6995     605.9974948210429$
6996     4702.522747095733$
6997    17207.737631060354$
6998     33203.79571648326$
6999     50584.16343037768$
Name: Max_Transaction_Value, Length: 7000, dtype: object
0        6525.814860765247$
1        2186.742245256916$
2       2743.4068078554915$
3         4360.78499399125$
4        4532.872520080136$
               ...         
6995     80.35266634214777$
6996    359.51133234175296$
6997    1510.5983293698946$
6998    1976.32612

In [257]:
for column in columns_to_change:
  df[column] = df[column].str.replace('$', '', regex=False)

In [258]:
for column in columns_to_change:
  print(df[column][df[column].str.contains(regex, na=False)])

Series([], Name: Avg_Transaction_Value, dtype: object)
Series([], Name: Max_Transaction_Value, dtype: object)
Series([], Name: Min_Transaction_Value, dtype: object)


In [259]:
df.head()

,Age,Income_Level,Total_Transactions,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Total_Spent,Active_Days,Last_Transaction_Days_Ago,Loyalty_Points_Earned,Referral_Count,Cashback_Received,App_Usage_Frequency,Preferred_Payment_Method,Support_Tickets_Raised,Issue_Resolution_Time,Customer_Satisfaction_Score,LTV
0,54,Low,192,16736.38402319893,60216.83450988285,6525.814860765247,3.213386e+06,140,209,2114.0,25,2224.012140,Monthly,Debit Card,3,61.568590,1,3.279546e+05
1,67,High,979,14536.734682848608,48350.10027249861,2186.742245256916,1.423146e+07,229,240,2960.0,20,4026.823518,Monthly,UPI,17,60.392889,8,1.437053e+06
2,44,High,329,7061.372800216672,32521.157187328816,2743.4068078554915,NaN,73,21,3170.0,0,1441.011395,NaN,Debit Card,11,45.305579,4,2.419387e+05
3,30,High,71,16426.876453277957,17827.896720459485,4360.78499399125,NaN,299,285,NaN,35,4365.855580,NaN,Wallet Balance,6,22.030191,1,1.284599e+05
4,58,Middle,878,10800.09265965521,17497.634533762135,4532.872520080136,9.482481e+06,236,329,1992.0,18,4161.523827,Daily,UPI,18,20.634723,5,9.569514e+05


In [260]:
for column in columns_to_change:
  df[column] = df[column].astype(float)

In [261]:
df.head()

,Age,Income_Level,Total_Transactions,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Total_Spent,Active_Days,Last_Transaction_Days_Ago,Loyalty_Points_Earned,Referral_Count,Cashback_Received,App_Usage_Frequency,Preferred_Payment_Method,Support_Tickets_Raised,Issue_Resolution_Time,Customer_Satisfaction_Score,LTV
0,54,Low,192,16736.384023,60216.834510,6525.814861,3.213386e+06,140,209,2114.0,25,2224.012140,Monthly,Debit Card,3,61.568590,1,3.279546e+05
1,67,High,979,14536.734683,48350.100272,2186.742245,1.423146e+07,229,240,2960.0,20,4026.823518,Monthly,UPI,17,60.392889,8,1.437053e+06
2,44,High,329,7061.372800,32521.157187,2743.406808,NaN,73,21,3170.0,0,1441.011395,NaN,Debit Card,11,45.305579,4,2.419387e+05
3,30,High,71,16426.876453,17827.896720,4360.784994,NaN,299,285,NaN,35,4365.855580,NaN,Wallet Balance,6,22.030191,1,1.284599e+05
4,58,Middle,878,10800.092660,17497.634534,4532.872520,9.482481e+06,236,329,1992.0,18,4161.523827,Daily,UPI,18,20.634723,5,9.569514e+05


In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          7000 non-null   int64  
 1   Income_Level                 6510 non-null   object 
 2   Total_Transactions           7000 non-null   int64  
 3   Avg_Transaction_Value        7000 non-null   float64
 4   Max_Transaction_Value        7000 non-null   float64
 5   Min_Transaction_Value        7000 non-null   float64
 6   Total_Spent                  5950 non-null   float64
 7   Active_Days                  7000 non-null   int64  
 8   Last_Transaction_Days_Ago    7000 non-null   int64  
 9   Loyalty_Points_Earned        6370 non-null   float64
 10  Referral_Count               7000 non-null   int64  
 11  Cashback_Received            7000 non-null   float64
 12  App_Usage_Frequency          5460 non-null   object 
 13  Preferred_Payment_

тепер треба почитисти викиди і nan

In [263]:
# numeric_columns = np.array([column for column in df.columns if df[column].dtype == 'float64'], dtype='object')
numeric_columns = df.select_dtypes(include=[float, int]).columns

# ітерація по колонкаї
for column in numeric_columns:
  print(f'# {column}')
  # встановлення нижнього та верхнього квантилів
  lower_bound = df[column].quantile(0.05)
  upper_bound = df[column].quantile(0.95)

  # виведення кількості рядків до видалення викидів
  print(f'Кількість рядків до видалення викидів {column} : {df.shape[0]}')

  # створення очищеного фрейму, який міститиме дані з колонок
  # тільки, якщо вони більші за нижній квантиль ТА нижчі за верхній
  # АБО має значення NaN
  df_cleaned = df[((df[column] >= lower_bound) & (df[column] <= upper_bound)) | (df[column].isna())]

  # виведення кількості рядків після видалення викидів
  # та кількість видалених рядків
  print(f'Кількість рядків після видалення викидів {column} : {df_cleaned.shape[0]}')
  print(f'Видалено рядків з {column} : {df.shape[0] - df_cleaned.shape[0]}')
  print()

# Age
Кількість рядків до видалення викидів Age : 7000
Кількість рядків після видалення викидів Age : 6474
Видалено рядків з Age : 526

# Total_Transactions
Кількість рядків до видалення викидів Total_Transactions : 7000
Кількість рядків після видалення викидів Total_Transactions : 6303
Видалено рядків з Total_Transactions : 697

# Avg_Transaction_Value
Кількість рядків до видалення викидів Avg_Transaction_Value : 7000
Кількість рядків після видалення викидів Avg_Transaction_Value : 6300
Видалено рядків з Avg_Transaction_Value : 700

# Max_Transaction_Value
Кількість рядків до видалення викидів Max_Transaction_Value : 7000
Кількість рядків після видалення викидів Max_Transaction_Value : 6300
Видалено рядків з Max_Transaction_Value : 700

# Min_Transaction_Value
Кількість рядків до видалення викидів Min_Transaction_Value : 7000
Кількість рядків після видалення викидів Min_Transaction_Value : 6300
Видалено рядків з Min_Transaction_Value : 700

# Total_Spent
Кількість рядків до видалення 

In [264]:
df = df_cleaned

In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6300 entries, 0 to 6999
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          6300 non-null   int64  
 1   Income_Level                 5854 non-null   object 
 2   Total_Transactions           6300 non-null   int64  
 3   Avg_Transaction_Value        6300 non-null   float64
 4   Max_Transaction_Value        6300 non-null   float64
 5   Min_Transaction_Value        6300 non-null   float64
 6   Total_Spent                  5351 non-null   float64
 7   Active_Days                  6300 non-null   int64  
 8   Last_Transaction_Days_Ago    6300 non-null   int64  
 9   Loyalty_Points_Earned        5730 non-null   float64
 10  Referral_Count               6300 non-null   int64  
 11  Cashback_Received            6300 non-null   float64
 12  App_Usage_Frequency          4900 non-null   object 
 13  Preferred_Payment_Metho

In [266]:
df = df.dropna(thresh = len(df) * 0.5, axis=1)
df.info() # виведемо інформацію для порівняння

<class 'pandas.core.frame.DataFrame'>
Index: 6300 entries, 0 to 6999
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          6300 non-null   int64  
 1   Income_Level                 5854 non-null   object 
 2   Total_Transactions           6300 non-null   int64  
 3   Avg_Transaction_Value        6300 non-null   float64
 4   Max_Transaction_Value        6300 non-null   float64
 5   Min_Transaction_Value        6300 non-null   float64
 6   Total_Spent                  5351 non-null   float64
 7   Active_Days                  6300 non-null   int64  
 8   Last_Transaction_Days_Ago    6300 non-null   int64  
 9   Loyalty_Points_Earned        5730 non-null   float64
 10  Referral_Count               6300 non-null   int64  
 11  Cashback_Received            6300 non-null   float64
 12  App_Usage_Frequency          4900 non-null   object 
 13  Preferred_Payment_Metho

In [267]:
numeric_columns = df.select_dtypes([float, int]).columns
numeric_columns

Index(['Age', 'Total_Transactions', 'Avg_Transaction_Value',
       'Max_Transaction_Value', 'Min_Transaction_Value', 'Total_Spent',
       'Active_Days', 'Last_Transaction_Days_Ago', 'Loyalty_Points_Earned',
       'Referral_Count', 'Cashback_Received', 'Support_Tickets_Raised',
       'Issue_Resolution_Time', 'Customer_Satisfaction_Score', 'LTV'],
      dtype='object')

In [268]:
for column in numeric_columns:
  df.loc[:, column] = df[column].fillna(df.groupby('Age')[column].transform('mean'))

In [269]:
df.isnull().sum()

,0
Age,0
Income_Level,446
Total_Transactions,0
Avg_Transaction_Value,0
Max_Transaction_Value,0
Min_Transaction_Value,0
Total_Spent,0
Active_Days,0
Last_Transaction_Days_Ago,0
Loyalty_Points_Earned,0


In [270]:
for column in ['Income_Level', 'App_Usage_Frequency']:
  df.loc[:, column] = df[column].fillna(df[column].mode()[0])

матриці та їх перетворення

In [271]:
X = df.drop(['LTV'], axis=1)

y = df['LTV']

In [272]:
categorical_columns = X.select_dtypes(include=['object']).columns

numerical_columns = X.select_dtypes(include=[float, int]).columns

In [273]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

standardized_columns = scaler.fit_transform(X[numerical_columns])

standardized_X = pd.DataFrame(standardized_columns, columns=numerical_columns, index=X.index)

X = X.drop(columns=numerical_columns)

X = pd.concat([X, standardized_X], axis=1)

X.head()

,Income_Level,App_Usage_Frequency,Preferred_Payment_Method,Age,Total_Transactions,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Total_Spent,Active_Days,Last_Transaction_Days_Ago,Loyalty_Points_Earned,Referral_Count,Cashback_Received,Support_Tickets_Raised,Issue_Resolution_Time,Customer_Satisfaction_Score
0,Low,Monthly,Debit Card,0.729520,-1.128009,1.254409,1.433746,1.705491,-0.441495,-0.406211,0.236785,-0.295572,0.013842,-0.184449,-1.168429,1.229388,-1.571932
2,High,Weekly,Debit Card,0.087130,-0.621682,-0.517719,0.120452,-0.099668,0.027229,-1.043523,-1.545908,0.473360,-1.700047,-0.726903,0.160590,0.431562,-0.525085
3,High,Weekly,Wallet Balance,-0.812215,-1.575204,1.197718,-0.576284,0.672228,0.049317,1.106215,0.957448,-0.029525,0.699397,1.299394,-0.670047,-0.710274,-1.571932
4,Middle,Daily,UPI,0.986476,1.407326,0.167086,-0.591944,0.754357,1.428817,0.506952,1.374674,-0.384407,-0.466047,1.157835,1.323481,-0.778733,-0.176136
5,Middle,Daily,UPI,-1.261888,0.900998,-0.695061,-0.411126,-0.421223,-0.053163,0.116955,-0.863175,0.517777,-1.357269,0.074225,-0.670047,-0.945538,1.568608


In [274]:
X.isnull().sum()

,0
Income_Level,0
App_Usage_Frequency,0
Preferred_Payment_Method,0
Age,0
Total_Transactions,0
Avg_Transaction_Value,0
Max_Transaction_Value,0
Min_Transaction_Value,0
Total_Spent,0
Active_Days,0


In [275]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, drop='first')

encoded_columns = encoder.fit_transform(X[categorical_columns])

encoded_X = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_columns))

X = X.drop(columns=categorical_columns)
X = pd.concat([X, encoded_X], axis=1)

X.head()

,Age,Total_Transactions,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Total_Spent,Active_Days,Last_Transaction_Days_Ago,Loyalty_Points_Earned,Referral_Count,...,Support_Tickets_Raised,Issue_Resolution_Time,Customer_Satisfaction_Score,Income_Level_Low,Income_Level_Middle,App_Usage_Frequency_Monthly,App_Usage_Frequency_Weekly,Preferred_Payment_Method_Debit Card,Preferred_Payment_Method_UPI,Preferred_Payment_Method_Wallet Balance
0,0.729520,-1.128009,1.254409,1.433746,1.705491,-0.441495,-0.406211,0.236785,-0.295572,0.013842,...,-1.168429,1.229388,-1.571932,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.087130,-0.621682,-0.517719,0.120452,-0.099668,0.027229,-1.043523,-1.545908,0.473360,-1.700047,...,0.160590,0.431562,-0.525085,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.812215,-1.575204,1.197718,-0.576284,0.672228,0.049317,1.106215,0.957448,-0.029525,0.699397,...,-0.670047,-0.710274,-1.571932,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.986476,1.407326,0.167086,-0.591944,0.754357,1.428817,0.506952,1.374674,-0.384407,-0.466047,...,1.323481,-0.778733,-0.176136,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,-1.261888,0.900998,-0.695061,-0.411126,-0.421223,-0.053163,0.116955,-0.863175,0.517777,-1.357269,...,-0.670047,-0.945538,1.568608,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [283]:
encoded_columns.shape

(6300, 7)

In [284]:
encoded_X.shape

(6300, 7)

In [285]:
X.shape

(6934, 21)

In [276]:
# X = X.dropna(thresh = len(X) * 0.5)

In [287]:
X = X.iloc[:-634]

In [288]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6300 entries, 0 to 6999
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Age                                      6300 non-null   float64
 1   Total_Transactions                       6300 non-null   float64
 2   Avg_Transaction_Value                    6300 non-null   float64
 3   Max_Transaction_Value                    6300 non-null   float64
 4   Min_Transaction_Value                    6300 non-null   float64
 5   Total_Spent                              6300 non-null   float64
 6   Active_Days                              6300 non-null   float64
 7   Last_Transaction_Days_Ago                6300 non-null   float64
 8   Loyalty_Points_Earned                    6300 non-null   float64
 9   Referral_Count                           6300 non-null   float64
 10  Cashback_Received                        6300 non-nul

In [289]:
X.tail()

,Age,Total_Transactions,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Total_Spent,Active_Days,Last_Transaction_Days_Ago,Loyalty_Points_Earned,Referral_Count,...,Support_Tickets_Raised,Issue_Resolution_Time,Customer_Satisfaction_Score,Income_Level_Low,Income_Level_Middle,App_Usage_Frequency_Monthly,App_Usage_Frequency_Weekly,Preferred_Payment_Method_Debit Card,Preferred_Payment_Method_UPI,Preferred_Payment_Method_Wallet Balance
6994,-1.711561,-1.711950,0.082253,-0.159639,-0.343653,-1.295318,-1.604737,-0.275265,0.672146,-1.425824,...,-0.670047,1.040961,-0.874034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6996,1.243432,0.553590,-1.522503,-1.198672,-1.237386,-1.096020,0.839876,0.644528,0.329185,1.042175,...,0.492844,-0.822855,-0.874034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6997,-1.711561,-0.059916,0.831841,-0.605691,-0.688028,0.670453,0.402319,1.554840,-0.070572,-1.151602,...,0.326717,0.418383,-1.222983,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6998,-0.683737,0.283795,-0.344874,0.152822,-0.465759,-0.058197,-0.586941,-1.270918,-1.373242,0.425175,...,0.160590,0.374664,-1.571932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6999,-1.647322,1.211447,1.064152,0.976977,2.329457,2.463483,0.383294,0.047137,-1.363048,-0.877380,...,1.157354,-0.068429,-1.571932,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
y_array = y.to_numpy().reshape(-1, 1)

target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y_array)

In [279]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6934 entries, 0 to 6296
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Age                                      6300 non-null   float64
 1   Total_Transactions                       6300 non-null   float64
 2   Avg_Transaction_Value                    6300 non-null   float64
 3   Max_Transaction_Value                    6300 non-null   float64
 4   Min_Transaction_Value                    6300 non-null   float64
 5   Total_Spent                              6300 non-null   float64
 6   Active_Days                              6300 non-null   float64
 7   Last_Transaction_Days_Ago                6300 non-null   float64
 8   Loyalty_Points_Earned                    6300 non-null   float64
 9   Referral_Count                           6300 non-null   float64
 10  Cashback_Received                        6300 non-nul

In [280]:
X.isnull().sum()

,0
Age,634
Total_Transactions,634
Avg_Transaction_Value,634
Max_Transaction_Value,634
Min_Transaction_Value,634
Total_Spent,634
Active_Days,634
Last_Transaction_Days_Ago,634
Loyalty_Points_Earned,634
Referral_Count,634


In [281]:
y_scaled.shape

(6300, 1)

In [290]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)